In [ ]:
import random
from typing import List, Tuple

In [ ]:
# Параметры алгоритма

# n = 10 # размерность векторов особей и матрицы стоимостей
# d = 50 # число векторов (такое кол-во мы должны поддерживать на протяжении всего ГА)
# # договариваемся брать только четные значения

# min_cost = 5 # минимальная стоимость назначения
# max_cost = 100 # максимальная стоимость назначения

# Тип для пары (стоимость или приспособленность, решение) (стоимость будет отрицательная, для решения задачи на MAX)
SolutionPair = Tuple[int, List[int]]

# Выбранная пара для кроссинговера
CrossingPair = Tuple[List[int], List[int]]

# # максимальное кол-во итераций
# MAX_ITER = 50

# # Значение итераций, при котором тоже произойдет выход из алгоритма, в случае неизменения максимального значения функции приспособленности
# MAX_ITER_WITH_MAX = 20

# # вероятность кроссинговера
# m = 0.9
# # вероятность мутации
# r = 0.05
# # количество перестановок в одной мутации
# mutn=1

In [ ]:
# вернуть 1 - если кроссинговер производится, 0 - иначе
def get_probability(m: float):
  return 1 if random.random() < m else 0

In [ ]:
# распечать популяцию с их значениями стоимостей
def print_population(population: List[SolutionPair], printnum: int):
  for solution in population[:printnum]:
    print("cost: ", solution[0], " ; decision ", " ".join([str(num) for num in solution[1]]), '\n')

In [ ]:
# генерация матрицы стоимостей
def generate_cost_matrix(n: int, min_cost: int, max_cost: int):
    return [[random.randint(min_cost, max_cost) for _ in range(n)] for _ in range(n)]

In [ ]:
# Генерация одного допустимого решения
def generate_valid_assignment(n: int):
    return random.sample(range(n), n)

In [ ]:
decision=generate_valid_assignment(5)
print(decision)

[2, 1, 3, 4, 0]


In [ ]:
# Генерация популяции допустимых решений
def generate_population(population_size: int, n: int):
    population = []

    for _ in range(population_size):
        population.append(generate_valid_assignment(n))

    return population

In [ ]:
population=generate_population(10, 5)
print(population)

[[3, 1, 0, 4, 2], [4, 1, 3, 2, 0], [1, 4, 2, 0, 3], [3, 2, 4, 0, 1], [3, 2, 4, 1, 0], [4, 2, 3, 1, 0], [4, 2, 3, 1, 0], [1, 2, 4, 0, 3], [1, 0, 2, 3, 4], [4, 2, 0, 3, 1]]


In [ ]:
# вывод матрицы
def print_matrix(matrix):
  for vec in matrix:
    row = " ".join([str(num) for num in vec])
    print(row, '\n')

In [ ]:
# вычисление приспособленности
# возвращаем инверсию (с отрицательным знаком, чтобы потом решать задачу на максимум)
def calculate_fitness(decision, cost_matrix):
  sum = 0
  for i in range(0, len(cost_matrix)):
    sum += cost_matrix[i][decision[i]]

  return (-1) * sum

In [ ]:
# создание пары
def generate_solution_pair(decision, cost_matrix):
  fitness = calculate_fitness(decision, cost_matrix)
  return (fitness, decision)

In [ ]:
# Вернуть Список, в котором располагаются пары - Стоимость (отрицательная) -> соответствующее допустимое решение
def generate_solution_pairs(population, cost_matrix) -> List[SolutionPair]:
  return [generate_solution_pair(decision, cost_matrix) for decision in population]

In [ ]:
# Провести ранжирование (сортировка) популяции на по убыванию
def sort_solution_pairs(solution_pairs: List[SolutionPair]):
    return sorted(solution_pairs, key=lambda x: x[0], reverse=True)

In [ ]:
# вернуть максимальное значение функции приспособленности
def get_max_value_fitness(sorted_solution_pairs: List[SolutionPair]):
  return sorted_solution_pairs[0][0]

In [ ]:
# осуществление кроссинговера
def make_crossingover(pair: CrossingPair, size):
  child = [None] * size
  left_parent = pair[0]
  right_parent = pair[1]

  # наследуем от родителей
  for i in range(size):
    if left_parent[i] == right_parent[i]:
      child[i] = left_parent[i]

  # Находим элементы, которые нужно разместить
  used_values = set(x for x in child if x is not None)
  all_values = set(range(size))
  remaining_values = list(all_values - used_values)

  random.shuffle(remaining_values)

  idx = 0

  for i in range(size):
    if child[i] is None:
      child[i] = remaining_values[idx]
      idx += 1

  return child

In [ ]:
# получить список пар для кроссинговера (передается отсортированная популяция)
def get_pairs_for_crossingover(sorted_solution_pairs: List[SolutionPair], right_boarder: float):
  pairs_for_cross = []
  left_border = 0
  right_border = (int) (len(sorted_solution_pairs)*right_boarder)

  if right_border % 2 == 1:
    right_border += 1

  sample = random.sample(range(right_border), right_border)
  count = 0
  while (count < right_border):
    pairs_for_cross.append((sorted_solution_pairs[sample[count]][1], sorted_solution_pairs[sample[count + 1]][1]))
    count += 2

  return pairs_for_cross

In [ ]:
# пробуем осуществить кроссинговер у пар с заданной вероятностью m и вернусть список пар со стоимостью и решением
def get_descendants_from_crossingover(pairs: List[CrossingPair], cost_matrix, m, kidsn) -> List[SolutionPair]:
  descendants = []
  for pair in pairs:
    for i in range(kidsn):
      if get_probability(m) == 1:
        descendants.append(generate_solution_pair(make_crossingover(pair, len(pair[0])), cost_matrix))

  return descendants

In [ ]:
# избавиться от ненужных особей
def remove_useless_and_return(sorted_population: List[SolutionPair], true_size):
  return sorted_population[:true_size]

In [ ]:
# мутация
def mutation(mutn: int, n: int, decision):
  for i in range(mutn):
    mas=random.sample(range(n), 2)
    temp=decision[mas[0]]
    decision[mas[0]]=decision[mas[1]]
    decision[mas[1]]=temp
  return decision
def mutall(mutn: int, n: int, population: List[SolutionPair], cost_matrix, r):
  for i in range(len(population)):
    if get_probability(r) == 1:
      population[i]=generate_solution_pair(mutation(mutn, n, population[i][1]), cost_matrix)
  return population

In [ ]:
mutation(1,5,[1,4,2,5,3])

[4, 1, 2, 5, 3]

In [ ]:
from copy import deepcopy
# # Реализация генетического алгоритма
# def nprocess_gen_algorythm(printnum: int, d: int, n: int, min_cost: int, max_cost: int, MAX_ITER: int, MAX_ITER_WITH_MAX: int, m: float, r: float, mutn: int, cost_matrix, kidsn: int, right_boarder: float):
#   # Пусть изначально лучшая приспособленность имеет такое значение
#   max_value_fitness = -9999999

#   # счетчик максимального значения функции приспособленности
#   counter_max_value = 1

#   # максимальная приспособленность
#   bestsp=Tuple[int, List[int]]

#   # получаем матрицу стоимостей
#   # cost_matrix = generate_cost_matrix(n, min_cost, max_cost)
#   # её вывод
#   # print("Матрица стоимостей:\n")
#   print_matrix(cost_matrix)

#   # генерация исходной популяции
#   initial_population = generate_population(d,n)
#   # получение пар (стоимость - решение)
#   solution_pairs = generate_solution_pairs(initial_population, cost_matrix)

#   # ранжирование
#   solution_pairs = sort_solution_pairs(solution_pairs)

#   new_max = get_max_value_fitness(solution_pairs)

#   if new_max > max_value_fitness:
#     max_value_fitness = new_max
#     bestsp=solution_pairs[0]
#     print(bestsp)
#   #####следующая строка была закоменчена
#   print_population(solution_pairs, printnum)

#   # начинаем осуществлять итерации
#   iter = 0
#   while iter < MAX_ITER and counter_max_value < MAX_ITER_WITH_MAX:
#     #####следующая строка была закоменчена
#     print("Iteration ", iter)

#     # осуществим кроссинговер
#     childs = get_descendants_from_crossingover(get_pairs_for_crossingover(solution_pairs, right_boarder), cost_matrix, m, kidsn)

#     # расширим выборку
#     solution_pairs.extend(childs)

#     #мутируем
#     solution_pairs=mutall(mutn, n, solution_pairs, cost_matrix, r)

#     # произвидем ранжирование
#     solution_pairs = sort_solution_pairs(solution_pairs)

#     # обрежем популяцию
#     solution_pairs = remove_useless_and_return(solution_pairs, d)

#     # распечатаем популяцию
#     #####следующая строка была закоменчена
#     print_population(solution_pairs, printnum)

#     new_max = get_max_value_fitness(solution_pairs)

#     # обновим максимум, если надо, иначе обнулим
#     if new_max > max_value_fitness:
#       max_value_fitness = new_max
#       bestsp=deepcopy(solution_pairs[0])
#       print(bestsp)
#       counter_max_value = 0
#     else:
#       counter_max_value += 1

#     iter += 1
#   #####следующая строка была закоменчена
#   print("Лучшая особь: ", bestsp)

#   return iter

In [ ]:
# Реализация генетического алгоритма
def process_gen_algorythm(n: int,
                          d: int,
                          MAX_ITER: int,
                          MAX_ITER_WITH_MAX: int,
                          m: float,
                          r: float,
                          mutn: int,
                          cost_matrix,
                          kidsn: int,
                          right_boarder: float,
                          min_cost: int,
                          max_cost: int,
                          current_population: List[SolutionPair]):
  # Пусть изначально лучшая приспособленность имеет такое значение
  max_value_fitness = -9999999

  # счетчик максимального значения функции приспособленности
  counter_max_value = 1

  # максимальная приспособленность
  bestsp=Tuple[int, List[int]]

  # получаем матрицу стоимостей
  # cost_matrix = generate_cost_matrix(n, min_cost, max_cost)
  # её вывод
  # print("Матрица стоимостей:\n")
  # print_matrix(cost_matrix)

  # генерация исходной популяции (если нужно)
  if current_population is None:
    initial_population = generate_population(d,n)

    # получение пар (стоимость - решение)
    solution_pairs = generate_solution_pairs(initial_population, cost_matrix)
  else:
    solution_pairs = current_population

  solution_pairs = sort_solution_pairs(solution_pairs)

  new_max = get_max_value_fitness(solution_pairs)

  if new_max > max_value_fitness:
    max_value_fitness = new_max
    bestsp=solution_pairs[0]
    # print(bestsp)
  #####следующая строка была закоменчена
  # print_population(solution_pairs, printnum)

  # начинаем осуществлять итерации
  iter = 0
  while iter < MAX_ITER and counter_max_value < MAX_ITER_WITH_MAX:
    #####следующая строка была закоменчена
    # print("Iteration ", iter)

    # осуществим кроссинговер
    childs = get_descendants_from_crossingover(get_pairs_for_crossingover(solution_pairs, right_boarder), cost_matrix, m, kidsn)

    # расширим выборку
    solution_pairs.extend(childs)


    #мутируем
    solution_pairs=mutall(mutn, n, solution_pairs, cost_matrix, r)

    # произвидем ранжирование
    solution_pairs = sort_solution_pairs(solution_pairs)

    # обрежем популяцию
    solution_pairs = remove_useless_and_return(solution_pairs, d)

    # распечатаем популяцию
    #####следующая строка была закоменчена
    # print_population(solution_pairs, printnum)

    new_max = get_max_value_fitness(solution_pairs)

    # обновим максимум, если надо
    if new_max > max_value_fitness:
      max_value_fitness = new_max
      bestsp=deepcopy(solution_pairs[0])
      # print(bestsp)
      counter_max_value = 0
    else:
      counter_max_value += 1

    iter += 1
  #####следующая строка была закоменчена
  # print("Лучшая особь: ", bestsp)
  return [bestsp[0], list(solution_pairs)] # вернем всю популяцию для будущей миграции

# **FOR ANALYSIS**

In [ ]:
# cost_matrix = generate_cost_matrix(40, 5, 100)
# print("Матрица стоимостей:\n")
# print_matrix(cost_matrix)

In [ ]:
# def set_diagonal_low_cost(cost_matrix, min_diag: int, max_diag: int):
#     n = len(cost_matrix)
#     for i in range(n):
#         cost_matrix[i][i] = random.randint(min_diag, max_diag)

#     return cost_matrix

# cost_matrix = set_diagonal_low_cost(cost_matrix, 1, 5)
# print("Матрица стоимостей:\n")
# print_matrix(cost_matrix)

In [ ]:
# def set_diagonal_low_cost(cost_matrix):
#     n = len(cost_matrix)
#     for i in range(n):
#         cost_matrix[i][i] = 1

#     return cost_matrix

# cost_matrix = set_diagonal_low_cost(cost_matrix)
# print("Матрица стоимостей:\n")
# print_matrix(cost_matrix)

In [ ]:
# # тест алгоритма
# nprocess_gen_algorythm(5,100,10,5,100,500,50,1,0.05,5, cost_matrix, 2, 0.5, 10, 20)

In [ ]:
# from collections import Counter
# import matplotlib.pyplot as plt
# import numpy as np


# # -------------------------------
# # ОСНОВНАЯ ФУНКЦИЯ АНАЛИЗА
# # -------------------------------
# def analyze_parameters():
#     """
#     Проводит однопараметрический анализ (OFAT) для функции process_gen_algorythm.
#     Для каждого параметра по очереди заменяет его значение на все возможные,
#     оставляя остальные на базовых значениях.
#     Для каждого значения вызывает функцию 100 раз, находит моду.
#     Выводит значения параметров по убыванию результативности (моды).
#     """

#     # Списки параметров
#     param_lists = {
#         # 'n': [10, 20, 40, 100, 200],
#         # 'n': [10],
#         'd': [20, 40, 60, 100, 200],
#         'MAX_ITER': [50, 100, 200, 500],
#         'MAX_ITER_WITH_MAX': [15, 30, 60, 100],
#         'r': [0.02, 0.05, 0.1],
#         'mutn': [20, 14, 11],
#         'kidsn': [1, 2, 3, 4],
#         'rb': [0.25, 0.5]
#     }

#     # Базовый набор параметров (соответствует вашему)
#     base_params = [60, 200, 60, 0.05, 14, 2, 0.5]
#     param_names = list(param_lists.keys())  # ['n', 'd', 'MAX_ITER', ...]

#     # Результаты: {имя_параметра: [(значение, мода, частота), ...]}
#     results = {param: [] for param in param_names}

#     print("НАЧАЛО АНАЛИЗА ПАРАМЕТРОВ\n")

#     # Для визуализации: храним моды в исходном порядке для гистограмм
#     original_order_modas = {}  # {param: [мода для val1, мода для val2, ...]}

#     # Перебираем каждый параметр
#     for idx, param_name in enumerate(param_names):
#         print(f"{'='*80}")
#         print(f"   АНАЛИЗ ПАРАМЕТРА: {param_name.upper()}")
#         print(f"   Базовое значение: {base_params[idx]}")
#         print(f"   Тестируемые значения: {param_lists[param_name]}")
#         print('='*80)

#         # Храним результаты в порядке исходного списка значений
#         modas_in_order = []

#         # Для каждого значения этого параметра
#         for test_value in param_lists[param_name]:
#             # Создаем копию базового набора
#             current_params = base_params.copy()
#             # Меняем только текущий параметр
#             current_params[idx] = test_value

#             # if param_name == 'n':
#             #   cust_matrix=generate_cost_matrix(current_params[idx], 5, 100)
#             # else:
#             #   cust_matrix=cost_matrix

#             # Вызываем функцию 100 раз
#             outcomes = []
#             for trial in range(100):
#                 result = process_gen_algorythm(40,current_params[0],current_params[1],current_params[2],1, current_params[3],current_params[4], cost_matrix, current_params[5],current_params[6])
#                 outcomes.append(result)

#             # # Находим моду (наиболее частое значение)
#             # counter = Counter(outcomes)
#             # mode_value, mode_count = counter.most_common(1)[0]

#             counter = Counter(outcomes)
#             max_freq = max(counter.values())
#             # Все значения с максимальной частотой
#             candidates = [val for val, freq in counter.items() if freq == max_freq]
#             # Выбираем наибольшее из них
#             mode_value = max(candidates)
#             mode_count = max_freq

#             # Сохраняем результат
#             modas_in_order.append(mode_value)
#             results[param_name].append((test_value, mode_value, mode_count))
#             print(f"   {param_name}={test_value:>4} → мода: {mode_value:>3} (×{mode_count}/20)")

#         # Сохраняем порядок мод для гистограммы
#         original_order_modas[param_name] = modas_in_order

#         # Сортируем по убыванию моды (результативности)
#         results[param_name].sort(key=lambda x: x[1], reverse=True)

#         print(f"\nТОП по результативности (мода) для {param_name}:")
#         for val, mode, count in results[param_name]:
#             print(f"    {val:>4} → {mode:>3} (×{count})")
#         print()


#     # -------------------------------
#     # ПОСТРОЕНИЕ ГИСТОГРАММ С ФИКСИРОВАННОЙ ШИРИНОЙ СТОЛБЦОВ
#     # -------------------------------
#     print("ПОСТРОЕНИЕ ГИСТОГРАММ")
#     fig, axes = plt.subplots(2, 4, figsize=(18, 10))
#     axes = axes.flatten()

#     for i, param_name in enumerate(param_names):
#         modas = original_order_modas[param_name]
#         min_moda = min(modas)
#         adjusted_modas = [m - min_moda for m in modas]  # Сдвиг к нулю
#         values = param_lists[param_name]

#         ax = axes[i]

#         # Индексы для равномерного расположения столбцов
#         x_positions = np.arange(len(values))  # [0, 1, 2, ...]
#         width = 0.6  # фиксированная ширина столбца (можно регулировать: 0.4–0.8)

#         bars = ax.bar(x_positions, adjusted_modas, width=width, color='skyblue', edgecolor='black', alpha=0.8)

#         # Подписываем ось X реальными значениями
#         ax.set_xticks(x_positions)
#         ax.set_xticklabels(values, rotation=45, ha='right')  # поворот для читаемости

#         ax.set_title(f'{param_name.upper()}\n(сдвиг: мин. мода = {min_moda})')
#         ax.set_xlabel('Значение параметра')
#         ax.set_ylabel('Мода - min_мода')
#         ax.grid(axis='y', linestyle='--', alpha=0.7)

#         # Подписываем высоту каждого столбца
#         for j, bar in enumerate(bars):
#             height = bar.get_height()
#             ax.text(bar.get_x() + bar.get_width() / 2., height + 0.1,
#                     f'{int(height)}', ha='center', va='bottom', fontsize=8)

#     # Убираем лишние оси
#     for i in range(len(param_names), len(axes)):
#         fig.delaxes(axes[i])

#     plt.tight_layout()
#     plt.suptitle("Гистограммы: Результативность по параметрам (Y = мода - min_мода)", fontsize=16, y=1.02)
#     plt.show()
#     # -------------------------------
#     # ИТОГОВАЯ ТАБЛИЦА — ВСЕ ПАРАМЕТРЫ
#     # -------------------------------
#     print("\n" + "="*90)
#     print("ИТОГОВЫЙ РЕЙТИНГ ПАРАМЕТРОВ ПО РЕЗУЛЬТАТИВНОСТИ (по убыванию моды)")
#     print("="*90)

#     for param in param_names:
#         print(f"\n{param.upper():<12} | {'Значение':<8} | {'Мода':<6} | {'Частота':<8}")
#         print("-" * 50)
#         for val, mode, count in results[param]:
#             print(f"{val:<12} | {mode:<8} | {count:<8}")

# # Запуск
# if __name__ == "__main__":
#     analyze_parameters()

In [ ]:
# from collections import Counter
# import matplotlib.pyplot as plt
# import numpy as np

# def set_diagonal_low_cost(cost_matrix):
#     n = len(cost_matrix)
#     for i in range(n):
#         cost_matrix[i][i] = 1

#     return cost_matrix


# # -------------------------------
# # ОСНОВНАЯ ФУНКЦИЯ: АНАЛИЗ ТОЛЬКО ПАРАМЕТРА n
# # -------------------------------
# def analyze_n_only_max_frequency():
#     """
#     Анализирует влияние параметра n на результативность process_gen_algorythm.
#     Все остальные параметры зафиксированы на базовом наборе.
#     Для каждого значения n вызывает функцию 100 раз.
#     Находит моду с приоритетом на наибольшее значение при равных частотах.
#     Строит гистограмму с фиксированной шириной столбцов и Y = мода - min_мода.
#     """

#     # Параметр n и его значения (в исходном порядке)
#     n_values = [10, 20, 40, 100, 200]

#     # Базовый набор параметров (остальные фиксированы)
#     base_params = [40, 200, 200, 60, 1, 0.02, 14, 'cost_matrix', 4, 0.5]

#     # Храним результаты: [(n_value, max_result, frequency_of_max), ...]
#     results = []
#     max_frequencies = []  # для гистограммы — частота максимума для каждого n в порядке n_values

#     print("АНАЛИЗ ВЛИЯНИЯ ПАРАМЕТРА n на результативность\n")

#     # Перебираем каждое значение n
#     for n_val in n_values:
#         # Создаем копию базового набора, заменяем только n
#         current_params = base_params.copy()
#         current_params[0] = n_val  # n — первый параметр (индекс 0)
#         current_params[6] = n_val//2+1
#         # 20 запусков
#         cost_matrix=generate_cost_matrix(n_val, 5, 100)
#         cost_matrix = set_diagonal_low_cost(cost_matrix)
#         current_params[7]=cost_matrix
#         outcomes = []
#         for trial in range(100):
#             result = process_gen_algorythm(*current_params)
#             outcomes.append(result)

#         # НАХОДИМ МАКСИМАЛЬНОЕ ЗНАЧЕНИЕ И ЕГО ЧАСТОТУ
#         max_result = max(outcomes)             # Наибольший результат из 20
#         freq_of_max = outcomes.count(max_result)  # Сколько раз он встретился

#         max_frequencies.append(freq_of_max)
#         results.append((n_val, max_result, freq_of_max))

#         print(f"  n={n_val:>4} → максимум: {max_result:>3} (встречается {freq_of_max}/100 раз)")

#     # Сортируем по убыванию частоты максимума
#     results.sort(key=lambda x: x[2], reverse=True)

#     print("\nТОП по результативности (частота наибольшего результата):")
#     for n_val, max_val, freq in results:
#         print(f"    n={n_val:>4} → максимум: {max_val:>3} (×{freq}/100)")

#     # -------------------------------
#     # ПОСТРОЕНИЕ ГИСТОГРАММЫ
#     # -------------------------------
#     print("\nПОСТРОЕНИЕ ГИСТОГРАММЫ: Частота максимума (Y) vs n (X в исходном порядке)")

#     x_positions = np.arange(len(n_values))  # [0, 1, 2, 3, 4]
#     width = 0.6  # фиксированная ширина столбцов

#     plt.figure(figsize=(10, 6))
#     bars = plt.bar(x_positions, max_frequencies, width=width, color='lightcoral', edgecolor='black', alpha=0.8)

#     # Подписи оси X — реальные значения n
#     plt.xticks(x_positions, n_values, rotation=45, ha='right')
#     plt.title("Влияние параметра n на частоту достижения максимального результата\n(из 100 запусков)", fontsize=14)
#     plt.xlabel("Значение n")
#     plt.ylabel("Частота максимального результата (из 100)")
#     plt.ylim(0, 20)  # фиксируем Y от 0 до 20
#     plt.grid(axis='y', linestyle='--', alpha=0.7)

#     # Подписи над столбцами
#     for bar, freq in zip(bars, max_frequencies):
#         height = bar.get_height()
#         plt.text(bar.get_x() + bar.get_width() / 2., height + 0.3,
#                  f'{freq}', ha='center', va='bottom', fontsize=10, fontweight='bold')

#     plt.tight_layout()
#     plt.show()

#     # -------------------------------
#     # ИТОГОВАЯ ТАБЛИЦА
#     # -------------------------------
#     print("\n" + "="*55)
#     print(" ИТОГОВЫЙ РЕЙТИНГ ПО n (по убыванию частоты максимума)")
#     print("="*55)
#     print(f"{'n':<8} | {'Максимум':<8} | {'Частота':<8}")
#     print("-" * 55)
#     for n_val, max_val, freq in results:
#         print(f"{n_val:<8} | {max_val:<8} | {freq:<8}")

# # Запуск
# if __name__ == "__main__":
#     analyze_n_only_max_frequency()

# **МИГРАЦИЯ - ПАРАЛЛЕЛЬНЫЙ ГА**

In [ ]:
import numpy as np

# Обмен между двумя популяциями.
# Используется круговая миграция.
# first_sub_population_max - заберем самую лучшую особь
# second_sub_population_min - заберем самую плохую и отдадим первой подпопуляции
# Также производим сортировку после обмена
# Возвращаем актуальный список подпопуляций
def exchange_between_two_sub_population_and_return(first_sub_population_max: List[SolutionPair],
                                                   second_sub_population_min: List[SolutionPair]):
  top_from_first = first_sub_population_max.pop(0)
  bad_from_second = second_sub_population_min.pop(len(second_sub_population_min) - 1)

  first_sub_population_max.append(bad_from_second)
  second_sub_population_min.append(top_from_first)

  return sort_solution_pairs(first_sub_population_max), sort_solution_pairs(second_sub_population_min)

def initialize_sub_population(count_of_sub_population: int, size_of_sub_population: int, cost_matrix_size: int):
    oned_array = np.arange(count_of_sub_population * cost_matrix_size)
    sub_populations = np.zeros((count_of_sub_population, size_of_sub_population, cost_matrix_size), dtype=int)
    for i in range(count_of_sub_population):
      sub_populations[i] = generate_population(size_of_sub_population, cost_matrix_size)

    return sub_populations.tolist()

def return_max_solution(sub_populations):
    max_value = -9999999999
    result = None
    for sub_population in sub_populations:
      curr_value = sub_population[0][0]
      if curr_value > max_value:
        max_value = curr_value
        result = sub_population[0]

    return result

In [ ]:
def process_migration(
                          n: int, # Размер матрицы стоимостей
                          d: int, # Размер популяции (в подпопуляциях)
                          MAX_ITER: int, # Максимальное кол-во итераций внутри подпопуляций (поколения)
                          MAX_ITER_WITH_MAX: int, # Кол-во итераций, когда наилучшее значение функции приспособленности
                          m: float, # вероятнорсть кроссинговера
                          r: float, # вероятность мутации
                          mutn: int, # кол-во мутаций у одной особи
                          kidsn: int, # кол-во детей
                          right_boarder: float,             # параметр Элитизма, сколько нужно убрать ненужных особей после ранжирования (%/100)
                          min_cost: int,                    # Минимальная стоимость назначения
                          max_cost: int,                    # Максимальная стоимость назначения
                          COUNT_OF_SUB_POPULATION: int,     # Кол-во подпопуляций
                          MIGRATION_MAX_ITER_WITH_MAX: int, # кол-во итераций с одним и тем же макисмальным значением среди всех подпопуляций
                          MIGRATION_MAX_ITER: int           # Максимальное кол-во итераций при миграции

):
  cost_matrix = generate_cost_matrix(n, min_cost, max_cost)
  # Пусть изначально лучшая приспособленность имеет такое значение
  max_value_fitness = -9999999

  # счетчик максимального значения функции приспособленности
  counter_max_value = 1

  # Порядок подпопуляций сохраняем
  initial_population = initialize_sub_population(COUNT_OF_SUB_POPULATION, d, n)

  sub_population = []

  for init in initial_population:
    sub_population.append(generate_solution_pairs(init, cost_matrix))

  print(sub_population)

  bestsp=Tuple[int, List[int]]

  # def process_gen_algorythm(n: int,
  #                         d: int,
  #                         MAX_ITER: int,
  #                         MAX_ITER_WITH_MAX: int,
  #                         m: float,
  #                         r: float,
  #                         mutn: int,
  #                         cost_matrix,
  #                         kidsn: int,
  #                         right_boarder: float,
  #                         min_cost: int,
  #                         max_cost: int,
  #                         current_population: List[SolutionPair]):

  iter = 0
  while iter < MIGRATION_MAX_ITER and counter_max_value < MIGRATION_MAX_ITER_WITH_MAX:

    # Пускай популяции поработают раздельно определенное кол-во итераций
    for i in range(len(sub_population)):
      sub_population[i] = process_gen_algorythm(n,
                                                d,
                                                MAX_ITER,
                                                MAX_ITER_WITH_MAX,
                                                m,
                                                r,
                                                mutn,
                                                cost_matrix,
                                                kidsn,
                                                right_boarder,
                                                min_cost,
                                                max_cost,
                                                sub_population[i])[1]

    # Выполним миграции
    for i in range(len(sub_population)):
      if i == len(sub_population) - 1:
        sub_population[i], sub_population[0] = exchange_between_two_sub_population_and_return(sub_population[i], sub_population[0])
      else:
        sub_population[i], sub_population[i + 1] = exchange_between_two_sub_population_and_return(sub_population[i], sub_population[i + 1])

    new_max = return_max_solution(sub_population)

    # обновим максимум, если надо, иначе обнулим
    if new_max[0] > max_value_fitness:
      max_value_fitness = new_max[0]
      bestsp=deepcopy(new_max)
      # print(bestsp)
      counter_max_value = 0
    else:
      counter_max_value += 1

    iter += 1

    print("Кол-во произведенных миграций ", iter)
    print("BEST === ", bestsp)


  print("Result === ", bestsp)
  print("Count of iterations === ", iter)



In [ ]:
# Тест миграции (параллельного ГА)
process_migration(n=10,
                  d=5,
                  MAX_ITER=10,
                  MAX_ITER_WITH_MAX=5,
                  m=0.7,
                  r=0.5,
                  mutn=1,
                  kidsn=1,
                  right_boarder=0.5,
                  min_cost=1,
                  max_cost=10,
                  COUNT_OF_SUB_POPULATION=4,
                  MIGRATION_MAX_ITER_WITH_MAX=3,
                  MIGRATION_MAX_ITER=6
)


[[(-61, [3, 9, 6, 0, 5, 4, 1, 8, 7, 2]), (-55, [7, 6, 5, 0, 8, 2, 3, 1, 9, 4]), (-65, [2, 3, 9, 8, 0, 4, 1, 6, 5, 7]), (-35, [9, 4, 6, 1, 7, 3, 2, 0, 5, 8]), (-60, [8, 9, 2, 1, 6, 7, 4, 5, 0, 3])], [(-56, [9, 8, 3, 5, 7, 6, 2, 4, 0, 1]), (-52, [9, 2, 6, 0, 5, 7, 8, 3, 1, 4]), (-54, [0, 1, 6, 7, 8, 3, 4, 5, 2, 9]), (-65, [8, 5, 6, 0, 7, 9, 1, 4, 3, 2]), (-61, [6, 0, 1, 3, 8, 4, 2, 7, 9, 5])], [(-62, [5, 6, 3, 0, 1, 7, 9, 8, 4, 2]), (-63, [7, 9, 6, 3, 4, 5, 8, 0, 2, 1]), (-58, [8, 3, 4, 7, 6, 1, 9, 5, 2, 0]), (-71, [0, 3, 4, 9, 2, 5, 8, 6, 7, 1]), (-51, [5, 1, 2, 4, 9, 0, 3, 8, 6, 7])], [(-43, [4, 7, 6, 1, 8, 0, 2, 9, 5, 3]), (-65, [0, 1, 7, 8, 4, 2, 9, 5, 3, 6]), (-55, [5, 2, 1, 9, 8, 3, 6, 4, 7, 0]), (-57, [9, 6, 0, 4, 2, 1, 8, 7, 3, 5]), (-56, [9, 0, 3, 6, 4, 5, 7, 8, 1, 2])]]
Кол-во произведенных миграций  1
BEST ===  (-41, [9, 4, 6, 7, 5, 0, 8, 3, 1, 2])
Кол-во произведенных миграций  2
BEST ===  (-41, [9, 4, 6, 7, 5, 0, 8, 3, 1, 2])
Кол-во произведенных миграций  3
BEST ===  (-41, 

In [ ]:
def process_migration_outer(
                          n: int, # Размер матрицы стоимостей
                          d: int, # Размер популяции (в подпопуляциях)
                          MAX_ITER: int, # Максимальное кол-во итераций внутри подпопуляций (поколения)
                          MAX_ITER_WITH_MAX: int, # Кол-во итераций, когда наилучшее значение функции приспособленности
                          m: float, # вероятнорсть кроссинговера
                          r: float, # вероятность мутации
                          mutn: int, # кол-во мутаций у одной особи
                          cost_matrix, #КОСТМАТРИКССС
                          kidsn: int, # кол-во детей
                          right_boarder: float,             # параметр Элитизма, сколько нужно убрать ненужных особей после ранжирования (%/100)
                          min_cost: int,                    # Минимальная стоимость назначения
                          max_cost: int,                    # Максимальная стоимость назначения
                          none,
                          COUNT_OF_SUB_POPULATION: int,     # Кол-во подпопуляций
                          MIGRATION_MAX_ITER_WITH_MAX: int, # кол-во итераций с одним и тем же макисмальным значением среди всех подпопуляций
                          MIGRATION_MAX_ITER: int           # Максимальное кол-во итераций при миграции

):
  # cost_matrix = generate_cost_matrix(n, min_cost, max_cost)
  # Пусть изначально лучшая приспособленность имеет такое значение
  max_value_fitness = -9999999

  # счетчик максимального значения функции приспособленности
  counter_max_value = 1

  # Порядок подпопуляций сохраняем
  initial_population = initialize_sub_population(COUNT_OF_SUB_POPULATION, d, n)

  sub_population = []

  for init in initial_population:
    sub_population.append(generate_solution_pairs(init, cost_matrix))

  # print(sub_population)

  bestsp=Tuple[int, List[int]]

  # def process_gen_algorythm(n: int,
  #                         d: int,
  #                         MAX_ITER: int,
  #                         MAX_ITER_WITH_MAX: int,
  #                         m: float,
  #                         r: float,
  #                         mutn: int,
  #                         cost_matrix,
  #                         kidsn: int,
  #                         right_boarder: float,
  #                         min_cost: int,
  #                         max_cost: int,
  #                         current_population: List[SolutionPair]):

  iter = 0
  while iter < MIGRATION_MAX_ITER and counter_max_value < MIGRATION_MAX_ITER_WITH_MAX:

    # Пускай популяции поработают раздельно определенное кол-во итераций
    for i in range(len(sub_population)):
      sub_population[i] = process_gen_algorythm(n,
                                                d,
                                                MAX_ITER,
                                                MAX_ITER_WITH_MAX,
                                                m,
                                                r,
                                                mutn,
                                                cost_matrix,
                                                kidsn,
                                                right_boarder,
                                                min_cost,
                                                max_cost,
                                                sub_population[i])[1]

    # Выполним миграции
    for i in range(len(sub_population)):
      if i == len(sub_population) - 1:
        sub_population[i], sub_population[0] = exchange_between_two_sub_population_and_return(sub_population[i], sub_population[0])
      else:
        sub_population[i], sub_population[i + 1] = exchange_between_two_sub_population_and_return(sub_population[i], sub_population[i + 1])

    new_max = return_max_solution(sub_population)

    # обновим максимум, если надо, иначе обнулим
    if new_max[0] > max_value_fitness:
      max_value_fitness = new_max[0]
      bestsp=deepcopy(new_max)
      # print(bestsp)
      counter_max_value = 0
    else:
      counter_max_value += 1

    iter += 1

    # print("Кол-во произведенных миграций ", iter)
    # print("BEST === ", bestsp)


  # print("Result === ", bestsp)
  # print("Count of iterations === ", iter)
  return bestsp[0]


In [ ]:
def set_diagonal_low_cost(cost_matrix):
    n = len(cost_matrix)
    for i in range(n):
        cost_matrix[i][i] = 1

    return cost_matrix
mmas=[]
mmas.append(set_diagonal_low_cost(generate_cost_matrix(10, 5, 100)))
mmas.append(set_diagonal_low_cost(generate_cost_matrix(12, 5, 100)))
mmas.append(set_diagonal_low_cost(generate_cost_matrix(14, 5, 100)))
mmas.append(set_diagonal_low_cost(generate_cost_matrix(16, 5, 100)))
mmas.append(set_diagonal_low_cost(generate_cost_matrix(18, 5, 100)))
mmas.append(set_diagonal_low_cost(generate_cost_matrix(20, 5, 100)))

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np


def analyze_n_only_max_frequency_and_migration(mmas):

    # Параметр n и его значения (в исходном порядке)
    n_values = [10, 12, 14,16,18,20]

    # Базовый набор параметров (остальные фиксированы)
    base_params = [40, 200, 200, 60, 1, 0.02, 14, 'cost_matrix', 4, 0.5,5,100,None]

    # Храним результаты: [(n_value, max_result, frequency_of_max), ...]
    results = []
    resultsm = []


    print("АНАЛИЗ ВЛИЯНИЯ ПАРАМЕТРА n на результативность\n")

    # Перебираем каждое значение n
    for n_val in n_values:
        # Создаем копию базового набора, заменяем только n
        current_params = base_params.copy()
        current_params[0] = n_val  # n — первый параметр (индекс 0)
        current_params[6] = n_val//2+1
        current_params[7]=mmas[n_values.index(n_val)]
        outcomes = []
        outcomesm = []
        for trial in range(20):
            result = process_gen_algorythm(*current_params)[0]
            outcomes.append(result)
        current_params[1]=int(current_params[1]/5)
        for trial in range(20):
            result = process_migration_outer(*current_params, 5,60,200)
            outcomesm.append(result)

        # НАХОДИМ МАКСИМАЛЬНОЕ ЗНАЧЕНИЕ И ЕГО ЧАСТОТУ
        max_result = max(outcomes)             # Наибольший результат из 20
        freq_of_max = outcomes.count(max_result)  # Сколько раз он встретился
        max_resultm = max(outcomesm)             # Наибольший результат из 20
        freq_of_maxm = outcomesm.count(max_resultm)  # Сколько раз он встретился


        results.append((n_val, max_result, freq_of_max))

        resultsm.append((n_val, max_resultm, freq_of_maxm))

        print(f"  n={n_val:>4} → максимум: {max_result:>3} (встречается {freq_of_max}/20 раз)")
        print(f"  n={n_val:>4} → мигромаксимум: {max_resultm:>3} (встречается {freq_of_maxm}/20 раз)")

    # # Сортируем по убыванию частоты максимума
    # results.sort(key=lambda x: x[2], reverse=True)

    # print("\nТОП по результативности (частота наибольшего результата):")
    # for n_val, max_val, freq in results:
    #     print(f"    n={n_val:>4} → максимум: {max_val:>3} (×{freq}/100)")
    # # -------------------------------
    # # ИТОГОВАЯ ТАБЛИЦА
    # # -------------------------------
    # print("\n" + "="*55)
    # print(" ИТОГОВЫЙ РЕЙТИНГ ПО n (по убыванию частоты максимума)")
    # print("="*55)
    # print(f"{'n':<8} | {'Максимум':<8} | {'Частота':<8}")
    # print("-" * 55)
    # for n_val, max_val, freq in results:
    #     print(f"{n_val:<8} | {max_val:<8} | {freq:<8}")

# Запуск
if __name__ == "__main__":
    analyze_n_only_max_frequency_and_migration(mmas)

АНАЛИЗ ВЛИЯНИЯ ПАРАМЕТРА n на результативность

  n=  10 → максимум: -10 (встречается 19/20 раз)
  n=  10 → мигромаксимум: -10 (встречается 20/20 раз)
  n=  12 → максимум: -12 (встречается 20/20 раз)
  n=  12 → мигромаксимум: -12 (встречается 19/20 раз)
  n=  14 → максимум: -14 (встречается 9/20 раз)
  n=  14 → мигромаксимум: -14 (встречается 12/20 раз)
  n=  16 → максимум: -16 (встречается 2/20 раз)
  n=  16 → мигромаксимум: -16 (встречается 2/20 раз)
  n=  18 → максимум: -68 (встречается 1/20 раз)
  n=  18 → мигромаксимум: -76 (встречается 1/20 раз)
  n=  20 → максимум: -63 (встречается 1/20 раз)
  n=  20 → мигромаксимум: -128 (встречается 1/20 раз)
